In [2]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr

In [3]:
dataTrain = pd.read_csv('E:\\work\\kaggle\\data\\tecent\\pre\\train.csv')
dataTest = pd.read_csv('E:\\work\\kaggle\\data\\tecent\\pre\\test.csv')
dataCreative = pd.read_csv('E:\\work\\kaggle\\data\\tecent\\pre\\ad.csv')
dataTrain = pd.merge(dataTrain, dataCreative, on='creativeID')
dataTest = pd.merge(dataTest, dataCreative, on='creativeID')

In [4]:
# 观察训练集和测试的重合程度
def overlap(field):
    trainCount = dataTrain[field].drop_duplicates().count();
    testCount = dataTest[field].drop_duplicates().count();
    allCount = dataTrain[field].drop_duplicates().append(dataTest[field].drop_duplicates()).drop_duplicates().count()
    overlapCount = trainCount + testCount - allCount
    monopolizeTrain = trainCount - overlapCount
    monopolizeTest = testCount - overlapCount
    print "overlap:" + field
    print "trainCount:" + str(trainCount) + "|testCount:" + str(testCount) + "|allCount:" + str(allCount)
    print "overlap:" + str(overlapCount) + "|monopolizeTrain:" + str(monopolizeTrain) + "|monopolizeTest:" + str(monopolizeTest)

In [5]:
overlap('userID')

overlap:userID
trainCount:2595627|testCount:297466|allCount:2805118
overlap:87975|monopolizeTrain:2507652|monopolizeTest:209491


In [55]:
# 观察今日CVR与前N日CVR的相关性，即CVR的稳定性为几天
def cvr_by_time(field):
    return dataTrain[['label','date',field]].groupby([field,'date']).sum()/dataTrain[['label','date',field]].groupby([field,'date']).count()
# 计算不同N下当日CVR与前N日平均CVR的相关系数
def nAvg(x, N):
    avg = 0.02487299735
    # 第一种情况，之前无投放，直接用平均CVR代替
    if ((x[0], x[1]-1) in se.index) is False:
        return avg
    #第二种情况，之前有不到M天的投放，则用这些天数的平均值
    count = 0;
    amount = 0;
    for i in range(N):
        if ((x[0], x[1]-1-i) in se.index) is False:
            break
        count = count+1
        amount += se['label'][x[0], x[1]-1-i]
    return amount/count
def getPearsonr(N):
    # 计算不同N下的皮尔森相关系数
    for i in range(N):
        se['avgLastN'] = pd.Series(se.reset_index().apply(nAvg, args=(i+1,), axis=1).values, index=se.index)
        print i+1
        print pearsonr(se['label'], se['avgLastN'])
        print "##########"

In [56]:
# 先根据日期统计数据
dataTrain['date'] = dataTrain['clickTime'].apply(lambda x: int(x/10000))
se = cvr_by_time('appID')
getPearsonr(10)

1
(0.74168643491139674, 2.0340330838880036e-88)
##########
2
(0.64805943520373521, 6.8771428427641027e-61)
##########
3
(0.58318144937390537, 6.8484056559325451e-47)
##########
4
(0.58567573338813284, 2.264304853928902e-47)
##########
5
(0.61456626433850081, 2.9546972599292097e-53)
##########
6
(0.61970783280368691, 2.2829680965776192e-54)
##########
7
(0.61161242083289458, 1.2593551687478645e-52)
##########
8
(0.59943872087058203, 4.2299337076432025e-50)
##########
9
(0.59805923102281378, 8.0499816668861587e-50)
##########
10
(0.60204922361773794, 1.2409622846319068e-50)
##########
